In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical


In [4]:
# Define data paths
data_dir = 'C:/IA_SEM_3/TCS_Sem_3_IA1_IA2/TCS_Sem-3/IA_2/Covid19-dataset/train'
classes = ['Covid', 'Normal', 'Viral Pneumonia']
num_classes = len(classes)

In [5]:
# Define image size and batch size
img_width, img_height = 224, 224
batch_size = 32


In [6]:
# Load images and labels
data = []
labels = []

for class_index, class_name in enumerate(classes):
    class_dir = os.path.join(data_dir, class_name)
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
        image = tf.keras.preprocessing.image.img_to_array(image)
        data.append(image)
        labels.append(class_index)


# Convert lists to arrays
data = np.array(data, dtype="float32")
labels = np.array(labels)

# Convert labels to one-hot encoded format
labels = to_categorical(labels, num_classes=num_classes)

# Split the data into training and validation sets
train_data, val_data, train_labels, val_labels = train_test_split(data, labels, test_size=0.2, stratify=labels, random_state=42)


In [7]:
# Data augmentation for training images
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [8]:
# Validation and test data rescaling
val_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [9]:
# Flow data from directories
train_generator = train_datagen.flow_from_directory(
    train_data,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    train_data,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    val_data, # Replace with test data
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

ValueError: listdir: embedded null character in path

In [ ]:

# Load ResNet50 with pretrained weights
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))


In [ ]:
# Add Global Average Pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)


In [ ]:
# Add a fully connected layer
x = Dense(512, activation='relu')(x)

In [ ]:
# Add output layer for classification
output = Dense(3, activation='softmax')(x)

In [ ]:
# Create the model
model = Model(inputs=base_model.input, outputs=output)

In [ ]:
# Compile the model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=30,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size,
    callbacks=[early_stopping]
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)